In [2]:
import threading
import pandas as pd

# Read the available database options into a dataframe
db_options=pd.read_csv("Database Options.csv", sep=',')
# Read the installed base into a dataframe with a ',' separator
df = pd.read_csv("Install Base.csv", sep=',')

#Calculate the license type based on the Product description 
df["License type"] = df.apply(
    lambda row:
        "Processor" if row["Product Description"].find("Processor") > 0
        else "NUP" if row["Product Description"].find("Named") > 0
        else "Unknown", axis=1)

# Sum up the amount of licenses and the ARR which has the same name and license type

licenses=df[["Product Name", "Quantity", "Contract ARR","License type"]][df["License type"] != "Unknown"].groupby(
    ["Product Name","License type"], as_index=False)[["Quantity", "Contract ARR"]].sum()

#Select the licenses which are database options, meaning that the keywords listed in the db_options
#is part of the license name. The DB Option field will also indicate which license it is in the db_option
#dataframe by index of the license entry

licenses["DB_Option"]=licenses.apply(
    lambda row: pd.Series(list(map(
        lambda option,no:
            no if option in row["Product Name"] else -1, db_options["Keyword"], range(db_options.__len__())
        ))).max(),
    axis=1)

#If the license is a DB option, we can extract the list price of CPU and NUP license from the db_option dataframe
#Based on the license type we can fill up the List price column with the proper type of list price.

licenses["List Price CPU"]=licenses.apply(
    lambda row:
        db_options["List price CPU"][row["DB_Option"]] if row["DB_Option"]>0 else 0,
    axis=1)

licenses["List Price NUP"]=licenses.apply(
    lambda row:
        db_options["List Price NUP"][row["DB_Option"]] if row["DB_Option"]>0 
        else 0,
    axis=1)

licenses["License Name"]=licenses.apply(
    lambda row:
        db_options["License name"][row["DB_Option"]] if row["DB_Option"]>0
        else "Not a DB License",
    axis=1)

licenses["List Price"]=licenses.apply(
    lambda row:
        row["List Price CPU"] if row["License type"] == "Processor" else
        row["List Price NUP"] if row["License type"] == "NUP" else
        1,
    axis=1)

#The average discount can be calculated based on the ratio of the ARR/Quantity and the list price of the license type

licenses["Average Discount"]=licenses.apply(
    lambda row:
        (row["Contract ARR"]/row["Quantity"])/row["List Price"]-1 if row["List Price"]>1 else 0,
    axis=1)

# We create a new dataframe called license_printable from licenses, just to do the right ouptut formatting.

licenses_printable=licenses[licenses["DB_Option"]>0]
licenses_printable["Contract ARR"]=licenses_printable["Contract ARR"].map('${:,.0f}'.format)
licenses_printable["Average Discount"]=licenses_printable["Average Discount"].map('{:+.0%}'.format)

print()
print(licenses_printable[["Product Name","License type","Quantity","Contract ARR","Average Discount"]])
print('-'*100)
print("Total:{:>54,.0f}{:>13,.0f}{:+17.0%}".format(
    licenses[licenses["DB_Option"]>0]["Quantity"].max(),
    licenses[licenses["DB_Option"]>0]["Contract ARR"].sum(),
    licenses[licenses["DB_Option"]>0].apply(        # this calculates the weighted average. Weight is based on the ARR value
         lambda row:
             row["Contract ARR"]*row["Average Discount"]
          ,axis=1).sum()/licenses["Contract ARR"][licenses["DB_Option"]>0].sum())   
    )


                         Product Name License type  Quantity Contract ARR  \
0  Database Lifecycle Management Pack    Processor        22      $27,158   
1                    Diagnostics Pack    Processor        22      $16,974   
8  Oracle Database Enterprise Edition    Processor        22     $196,539   
9                         Tuning Pack    Processor        22      $11,316   

  Average Discount  
0             -90%  
1             -90%  
8             -81%  
9             -90%  
----------------------------------------------------------------------------------------------------
Total:                                                    22      251,987             -83%


C:\Users\BMOLNAR\AppData\Local\Temp\ipykernel_5328\587666737.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  licenses_printable["Contract ARR"]=licenses_printable["Contract ARR"].map('${:,.0f}'.format)
C:\Users\BMOLNAR\AppData\Local\Temp\ipykernel_5328\587666737.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  licenses_printable["Average Discount"]=licenses_printable["Average Discount"].map('{:+.0%}'.format)
